# Development Notebooks

## Create Points for StreamStats

Usage Notes:


Outputs:


by Seth Lawler: slawler@dewberry.com & Shane Putnam: sputnam@dewberry.com

In [1]:
import os
from osgeo import gdal, ogr,osr
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
from rasterio.mask import mask

#### Load the Stream Grid:

In [2]:
in_tif=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\rock_creek_clip.tif' #Load the stream grid raster which was maksed by the catchment polygon

### Specify the Catchment Outlet:

In [3]:
class StreamGrid(object):
    ''''''
    def __init__(self,path):
        self.path = path
        self.data = gdal.Open(path)
        self.band = self.data.GetRasterBand(1)
        self.array= self.band.ReadAsArray()

    def dataframe(self):
        '''Returns the flow accumulation grid as a pandas dataframe object'''
        df = pd.DataFrame(self.array)
        return df

    def transform(self,x,y):
        '''Method takes indices from dataframe or array and returns projected coordinates'''
        ul_x, x_dim, x_rotation, ul_y, y_rotation, y_dim = self.data.GetGeoTransform()
        x_coord = x * x_dim + ul_x + (x_dim / 2.)
        y_coord = y * y_dim + ul_y + (y_dim / 2.)
        coords = x_coord, y_coord
        return coords

In [4]:
def MoveUpstream(df, row, col, nogo_cells):
    '''Function searches surrounding cells in stream network grid to identify the
        grid cell immediately upstream. Output is index pair of upstream cell '''
    curr_cell_idxs = (row, col)
    
    current_cell= df[row][col]
    assert current_cell.values == 1
    surr_cells = []

    for i in range(-1,2):
        
        for j in range(-1,2):

            value = df[row + i][col+j] #Read Value of raster cell

            if value == 0:
                # if value is zero, no stream in this cell
                continue
                
            elif value == 1:
                # if value is 1, add to list
                surr_cells.append([row + i],[col+j])
                
            elif (row, col) in nogo_cells:
                continue
                
            else:
                print("Error?")
                
    return curr_cell_idxs, surr_cells

In [5]:
def UpstreamIterator(flowgrid, idx_n, idx_list,str_len):
    '''Algorithm created to iterate over steps to calculate streamline distance and find the
       uus point of interest (stopping point, defined by distance given in while expression in __main__) '''
    r,c = int(idx_n[0][0]),int(idx_n[1][0])
    idx_n = MoveUpstream(flowgrid.dataframe(),r,c)
    xypair = idx_n[0][0],idx_n[1][0]
    str_len += GetDistance(r,c, xypair[0], xypair[1], cellsize)
    usxy = flowgrid.transform(xypair[0],xypair[1])
    idx_list.append(usxy)
    return usxy, idx_n, str_len

In [6]:
sg = StreamGrid(in_tif)
df = sg.dataframe()
df.replace(255,0, inplace=True)
df.head(n=2)

,0,1,2,3,4,5,6,7,8,9,...,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Specify the pour point to set the start location of the search:

In [7]:
lat=1925315.186
lon=1616784.964
pp_crs=570

In [8]:
transform=sg.data.GetGeoTransform()
xOrigin, yOrigin, pixelWidth, pixelHeight = transform[0], transform[3], transform[1], -transform[5]
pix_x = int((lon - xOrigin) / pixelWidth)
pix_y = int((yOrigin - lat ) / pixelHeight)
pourpoint=[(pix_x, pix_y)]
print(pourpoint)

[(877, 1848)]


In [19]:
def MoveUpstream(df, stream_cell, nogo):
    row=stream_cell[0][0]
    col=stream_cell[0][1]      
    cell_value= df[row][col]
    assert cell_value == 1
                 
    nogo.append((row,col))
    stream_cell=[]

    for i in range(-1,2):
        
        for j in range(-1,2):

            value = df[row + i][col+j] #Read Value of raster cell
            
            if value == 0: # if value is zero, no stream in this cell
                continue
            
            elif value==1 and (row+i,col+j) not in nogo: # if value is 1, add to list
                stream_cell.append((row + i,col+j)) 
    return stream_cell, nogo

In [20]:
def FindConfluence(df, stream_cell, nogo):
    while len(stream_cell)==1:
        stream_cell, nogo=MoveUpstream(df, stream_cell, nogo)
    return stream_cell, nogo

In [51]:
nogo=[]
confl=[]
ppoints=[]
stream_cell=pourpoint

confl, nogo=FindConfluence(df, stream_cell, nogo)

nogo=confl+nogo
confl[0]
stream_cell, nogo=MoveUpstream(df, [confl[0]],nogo)
for i in range(2):
    stream_cell, nogo=MoveUpstream(df, stream_cell,nogo)
ppoint_temp=stream_cell

[(870, 1801)]

In [49]:
ppoints

[(870, 1801)]

In [ ]:
ppoints=ppoints+ppoint_temp    